In [8]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger
import utils
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.metrics import mean_squared_error

In [9]:
class LSTMModel(lightning.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate):
        super().__init__()
        self.lr = learning_rate
        self.lstm = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out[:,-1,:].squeeze(-1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self.criterion(out, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self.criterion(out, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer


In [10]:
def create_sequences(x, y, seq_length):
    xs = []
    ys = []
    for i in range(len(x) - seq_length - 1):
        _x = x[i : i + seq_length]
        _y = y[i+seq_length]
        xs.append(_x)
        ys.append(_y)
    return np.array(xs), np.array(ys)


In [11]:
df_train, df_test, df_valid = utils.load_cleaned_data()

class ClimateDataset(torch.utils.data.Dataset):
    def __init__(self, df, seq_length) -> None:
        super().__init__()
        self.df = df
        self.x = df.iloc[:, 1:].values
        self.y = df.iloc[:, 0].values
        self.sequences, self.targets = create_sequences(self.x, self.y, seq_length)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        x = self.sequences[index]
        y = self.targets[index]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

BATCH_SIZE = 32
SEQ_LENGTH = 2

dataloader_train = torch.utils.data.DataLoader(ClimateDataset(df_train, SEQ_LENGTH), batch_size=BATCH_SIZE, shuffle=False, num_workers=15)
dataloader_valid = torch.utils.data.DataLoader(ClimateDataset(df_valid, SEQ_LENGTH), batch_size=BATCH_SIZE, shuffle=False, num_workers=15)


d:\Skola\MIT\3.semester\ADAaML\ADAML-Project2\venv\lib\site-packages\torch\utils\data\dataloader.py:557: UserWarning: This DataLoader will create 15 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:

def predict(model, dataloader):
    preds = []
    true = []
    ds = ClimateDataset(df_test, SEQ_LENGTH)

    it = iter(dataloader)

    while True:
        try:
            x, y = next(it)
            pred = model.forward(x)
            preds.append(pred.detach().numpy())
            true.append(y.detach().numpy())
        except StopIteration:
            break
    return np.array(preds), np.array(true)


In [13]:
import os


learning_rates = [1e-2, 5e-3, 1e-3]
hidden_sizes = [16, 32, 64]
num_layers = [3, 5, 8]


for lr, hidden_size, num_layer in product(learning_rates, hidden_sizes, num_layers):
    print("lr:", lr, "hidden_size:", hidden_size, "num_layer:", num_layer)
    model = LSTMModel(3, hidden_size=hidden_size, num_layers=num_layer, learning_rate=lr)

    trainer = lightning.Trainer(
    max_epochs=1000,
    log_every_n_steps=1,
    callbacks=[
        EarlyStopping(monitor="val_loss", mode="min", patience=10),
        ],
    logger=TensorBoardLogger(f"rnn-logs",name="", version=f"RNN-lr={lr}, hidden_size={hidden_size}, num_layer={num_layer}"),
    accelerator='auto'
    )

    trainer.fit(model, dataloader_train, dataloader_valid)

    dataloader_test = torch.utils.data.DataLoader(ClimateDataset(df_test, SEQ_LENGTH), batch_size=1, shuffle=False, num_workers=1)
    preds, true = predict(model, dataloader_test)

    folder_path = f"logs/RNN-lr={lr} hidden_size={hidden_size} num_layer={num_layer}"
    os.makedirs(folder_path, exist_ok=True)

    mse = mean_squared_error(true, preds)
    print("Mean Squared Error:", mse)
    print("Root Mean Squared Error:", np.sqrt(mse))
    plt.plot(preds, label="pred")
    plt.plot(true, label="true")
    plt.xlabel('Sample')
    plt.ylabel('Predicted temperature (scaled)')
    plot_path = os.path.join(folder_path, f"RNN-lr={lr} hidden_size={hidden_size} num_layer={num_layer}.png")
    plt.savefig(plot_path)
    plt.legend()
    plt.show()

    stats_path = os.path.join(folder_path, f"RNN-lr={lr} hidden_size={hidden_size} num_layer={num_layer}.txt")
    with open(stats_path, 'w') as stats_file:
      stats_file.write(f"Mean Square Error: {mse}\n Root Mean Squared Error: {np.sqrt(mse)}")


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type    | Params
--------------------------------------
0 | lstm      | RNN     | 1.4 K 
1 | fc        | Linear  | 17    
2 | criterion | MSELoss | 0     
--------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.006     Total estimated model params size (MB)


lr: 0.01 hidden_size: 16 num_layer: 3


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\Skola\MIT\3.semester\ADAaML\ADAML-Project2\venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


RuntimeError: DataLoader worker (pid(s) 23332, 19760, 12208, 15756, 20428) exited unexpectedly